In [4]:
import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk, ImageDraw
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

class App:
    def __init__(self):
        # Initialize Tkinter FIRST
        self.root = tk.Tk()
        self.root.title("Emojify")
        self.root.geometry("1400x900+100+10")
        self.root.configure(bg='black')
        
        # Initialize image registry tied to the root window
        self.image_registry = {}
        
        # Load model and setup OpenCV
        self.setup_model()
        self.setup_opencv()
        
        # Build GUI
        self.build_gui()
        
        # Start video processing
        self.show_vid()
        self.show_vid2()
        
        self.root.mainloop()
    
    def setup_model(self):
        # Rebuild model architecture
        self.emotion_model = Sequential([
            Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
            Conv2D(64, (3,3), activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(0.25),
            Conv2D(128, (3,3), activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Conv2D(128, (3,3), activation='relu'),
            MaxPooling2D(pool_size=(2,2)),
            Dropout(0.25),
            Flatten(),
            Dense(1024, activation='relu'),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])
        self.emotion_model.load_weights('model.weights.h5')
    
    def setup_opencv(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        self.cap = cv2.VideoCapture(0)
        self.show_text = [4]  # Mutable list for default emotion (Neutral)
    
    def get_image(self, path, default_text=None, size=None):
        """Atomic image loading with root window reference"""
        if path not in self.image_registry:
            try:
                img = Image.open(path)
                if size:
                    img = img.resize(size)
                self.image_registry[path] = ImageTk.PhotoImage(img, master=self.root)
            except FileNotFoundError:
                size = size or (200, 100)
                placeholder = Image.new('RGB', size, color=(30, 30, 30))
                d = ImageDraw.Draw(placeholder)
                text = default_text or "Missing: " + path.split('/')[-1]
                d.text((10, 10), text, fill=(255, 255, 0))
                self.image_registry[path] = ImageTk.PhotoImage(placeholder, master=self.root)
        return self.image_registry[path]
    
    def build_gui(self):
        # Logo
        logo_img = self.get_image("logo.png", default_text="Emojify", size=(300, 150))
        heading = Label(self.root, image=logo_img, bg='black')
        heading.image = logo_img
        heading.pack()
        
        # Title
        heading2 = Label(self.root, text="Photo to Emoji", pady=20, 
                        font=('arial', 45, 'bold'), bg='black', fg='#CDCDCD')
        heading2.pack()
        
        # Video and emoji labels
        self.lmain = Label(self.root, padx=50, bd=10)
        self.lmain.place(x=50, y=250)
        
        self.lmain2 = Label(self.root, bd=10)
        self.lmain2.place(x=900, y=350)
        
        self.lmain3 = Label(self.root, bd=10, fg="#CDCDCD", bg='black')
        self.lmain3.place(x=960, y=250)
        
        # Quit button
        exitbutton = Button(self.root, text='Quit', fg="red", 
                           command=self.root.destroy, font=('arial', 25, 'bold'))
        exitbutton.pack(side=BOTTOM)
    
    def show_vid(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.resize(frame, (600, 500))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
            
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                roi_gray = gray[y:y+h, x:x+w]
                cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
                prediction = self.emotion_model.predict(cropped_img)
                maxindex = int(np.argmax(prediction))
                cv2.putText(frame, emotion_dict[maxindex], (x+20, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                self.show_text[0] = maxindex
            
            # Update video feed
            img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            imgtk = ImageTk.PhotoImage(image=img, master=self.root)
            self.lmain.imgtk = imgtk
            self.lmain.configure(image=imgtk)
        
        self.root.after(10, self.show_vid)
    
    def show_vid2(self):
        emoji_path = emoji_dist.get(self.show_text[0], "./emojis/neutral.png")
        emoji_img = self.get_image(emoji_path, size=(200, 200))
        self.lmain2.configure(image=emoji_img)
        self.lmain3.configure(text=emotion_dict.get(self.show_text[0], "Neutral"), 
                             font=('arial', 45, 'bold'))
        self.root.after(100, self.show_vid2)

# Emotion and emoji mappings (global)
emotion_dict = {
    0: "Angry", 1: "Disgusted", 2: "Fearful",
    3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"
}

emoji_dist = {
    0: "./emojis/angry.png",
    1: "./emojis/disgusted.png",
    2: "./emojis/fearful.png",
    3: "./emojis/happy.png",
    4: "./emojis/neutral.png",
    5: "./emojis/sad.png",
    6: "./emojis/surprised.png"
}

if __name__ == '__main__':
    App()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━